# MODELO PREDITO - FRAUDE BANCÁRIA

O objetivo desta análise preditiva é determinar se determinada transação bancária tem potencial ou não de ser uma fraude. Dessa forma, o modelo auxilia em alertas sobre transações suspeitas e que podem impedir algum tipo de dano ao cliente e ao banco em si. O objetivo desse modelo é ter uma acurácia acima de 80%.

In [1]:
# carregando os pacotes e removendo os avisos
import numpy as np
import pandas as pd
import seaborn as sns
import scikitplot as skplt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import pylab
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# Carregando o dataset
base = pd.read_csv('C:/python_scripts/fraud/base.csv')

In [3]:
# Conferindo a quantidade de linhas e confirmando que a junção dos dataset está correta
print(base.shape)

(1000000, 32)


## 1. Análise Exploratória

Dataset possui 1 milhão de registros com 32 atributos. Como é um problema de classificação, é esperado que exista um desbalanceamento de classes e também possíveis atributos que estejam correlacionados. A fase de análise exploratória será excelente para identificar esses pontos e d